In [1]:
import numpy as np
import pandas as pd

## Household details: income, assets, expenditure, occupants, value, rent/mortgage

In [25]:
demographic_file = '../nta/demo_2021acs5yr_nta.xlsx'
economic_file = '../nta/econ_2021acs5yr_nta.xlsx'
social_file = '../nta/soc_2021acs5yr_nta.xlsx'
housing_file = '../nta/hous_2021acs5yrr_nta.xlsx'

In [38]:
income_mapping = {'U25': ['HHIU10', 'HHI10t14', 'HHI15t24'], '25t49': ['HHI25t34', 'HHI35t49'],
'50t99': ['HHI50t74', 'HHI75t99'], '100t199': ['HI100t149', 'HI150t199'], '200A': ['HHI200pl']}

ownership_mapping = {'renter': ['ROcHU1'], 'no_mortgage_owner': ['HUnoMrtg1'], 'mortgage_owner': ['HUwMrtg']}

## helpers

In [39]:
def compute_stats(nta_df, attr, estimation_vars=['E', 'M', 'C', 'P', 'Z'], filter=True):
    if filter:
        filter_vars = ['E', 'P']
        estimation_vars = filter_vars
    stats = [nta_df['{}{}'.format(attr, var)].values for var in estimation_vars]
    
    return stats

def merge_nta_stats(nta_df, attr_vals):
    '''estimate and percentage'''
    args = [compute_stats(nta_df, attr, filter=True) for attr in attr_vals]
    ret_e, ret_p = 0, 0
    for val in args:
        e, p = val
        ret_e += e
        ret_p += p
    
    return [ret_e, ret_p]

## processing

In [29]:
# occupants from social file
social_df = pd.read_excel(social_file)
# age, gender, race from demographic file
demo_df = pd.read_excel(demographic_file)
# income from economic file
econ_df = pd.read_excel(economic_file)
# ownership, mortgage/rent expense from household file
house_df = pd.read_excel(housing_file)

In [40]:
def get_nta_household_income(df, nta_id, income_mapping):
    nta_df = df[df['GeoID'] == nta_id]
    
    nta_household_income = {}

    for band in income_mapping:
        nta_household_income[band] = {}
        estimate, percentage = merge_nta_stats(nta_df, income_mapping[band])

        nta_household_income[band]['estimate'] = estimate
        nta_household_income[band]['probability'] = percentage / 100.0

    return nta_household_income

def get_nta_household_ownership(df, nta_id, ownership_mapping):
    nta_df = df[df['GeoID'] == nta_id]
    
    nta_household_ownership = {}

    # compute estimates
    for ownership_type in ownership_mapping:
        nta_household_ownership[ownership_type] = {}
        estimate, percentage = merge_nta_stats(nta_df, ownership_mapping[ownership_type])

        nta_household_ownership[ownership_type]['estimate'] = estimate

    all_households = sum([nta_household_ownership[o_type]['estimate'] for o_type in ownership_mapping])
    
    # compute probability
    for ownership_type in ownership_mapping:
        nta_household_ownership[ownership_type]['probability'] = nta_household_ownership[ownership_type]['estimate'] / all_households

    return nta_household_ownership

In [41]:
NTA_ID = 'BK0101'

In [42]:
nta_household_income = get_nta_household_income(econ_df, NTA_ID, income_mapping)
nta_household_ownership = get_nta_household_ownership(house_df, NTA_ID, ownership_mapping)

In [43]:
income_prob = [nta_household_income[key_ix]['probability'] for key_ix in nta_household_income]
ownership_prob = [nta_household_ownership[key_ix]['probability'] for key_ix in nta_household_ownership]

sum(income_prob), sum(ownership_prob)

(array([0.999]), array([1.]))

In [44]:
print(sum([nta_household_income[key_ix]['estimate'] for key_ix in nta_household_income]))

print(sum([nta_household_ownership[key_ix]['estimate'] for key_ix in nta_household_ownership]))

[17487]
[17487]
